# LangChain: Q&A over Documents

 Original Source: [LangChain for LLM Application Development](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer) 

In [8]:
# !pip install --upgrade langchain openai docarray

In [9]:
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown

In [17]:
# OpenAI API KEY
api_key = '?'

In [11]:
# File references
src_file = 'dataset/winemag-data-130k-v2.csv'
dest_file = 'dataset/wines_30.csv'
df = pd.read_csv(src_file).iloc[:30, 1:].drop(columns=[
    'designation',
    'points',
    'price',
    'province',
    'region_1',
    'region_2',
    'taster_name',
    'taster_twitter_handle'])
df.to_csv(dest_file)
df.head()

,country,description,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [12]:
# Initialize Vector Store Index
llm = OpenAI(temperature=0, openai_api_key=api_key)
loader = CSVLoader(file_path=dest_file)
embedding = OpenAIEmbeddings(openai_api_key=api_key)
index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [13]:
# Query
query = "List items from country Italy in a table in markdown with columns title, description and variety."
response = index.query(query, llm=llm)
display(Markdown(response))

 | Title | Description | Variety |
| --- | --- | --- |
| Terre di Giurfo 2013 Belsito Frappato (Vittoria) | Here's a bright, informal red that opens with aromas of candied berry, white pepper and savory herb that carry over to the palate. It's balanced with fresh acidity and soft tannins. | Frappato |
| Stemmari 2013 Dalila White (Terre Siciliane) | Pretty aromas of yellow flower and stone fruit lead the nose. The bright palate offers yellow apple, apricot, vanilla and delicate notes of lightly toasted oak alongside crisp acidity. | White Blend |
| Stemmari 2013 Nero d'Avola (Terre Siciliane) | Aromas recall ripe dark berry, toast and a whiff of cake spice. The soft, informal palate offers sour cherry, vanilla and a hint of espresso alongside round tannins. Drink soon. | Nero d'Avola |
| Terre di Giurfo 2011 Mascaria Barricato  (Cerasuolo di Vittoria) | Aromas suggest mature berry, scorched earth, animal, toast and

## Using RetrievalQA chain and ChatOpenAI model

In [14]:
# initialize LLM
llm = ChatOpenAI(temperature=0, openai_api_key=api_key)

# Initialize documents
dest_file = 'dataset/wines_30.csv'
loader = CSVLoader(file_path=dest_file)
docs = loader.load()

# initialize embeddings
embedding = OpenAIEmbeddings(openai_api_key=api_key)

# initialize db 
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embedding
)

# initialize retriever
retriever = db.as_retriever()

# initialize chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", # map_reduce, refine, map_rerank
    retriever=retriever, 
    verbose=True
)

In [15]:
# Query
query = "List items from country Spain and/or Argentina in a table in markdown with columns; country, title, description and variety."
response = qa_chain.run(query)
display(Markdown(response))



> Entering new RetrievalQA chain...

> Finished chain.


| Country | Title | Description | Variety |
|---------|-------|-------------|---------|
| Spain   | Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra) | Blackberry and raspberry aromas show a typical Navarran whiff of green herbs and, in this case, horseradish. In the mouth, this is fairly full bodied, with tomatoey acidity. Spicy, herbal flavors complement dark plum fruit, while the finish is fresh but grabby. | Tempranillo-Merlot |
| Argentina | Felix Lavaque 2010 Felix Malbec (Cafayate) | Baked plum, molasses, balsamic vinegar and cheesy oak aromas feed into a palate that's braced by a bolt of acidity. A compact set of saucy red-berry and plum flavors features tobacco and peppery accents, while the finish is mildly green in flavor, with respectable weight and balance. | Malbec |
| Argentina | Gaucho Andino 2011 Winemaker Selection Malbec (Mendoza) | Raw black-cherry aromas are direct and simple but good. This has a juicy feel that thickens over time, with oak character and extract becoming more apparent. A flavor profile driven by dark-berry fruits and smoldering oak finishes meaty but hot. | Malbec |
| Spain   | Pradorey 2010 Vendimia Seleccionada Finca Valdelayegua Single Vineyard Crianza  (Ribera del Duero) | Desiccated blackberry, leather, charred wood and mint aromas carry the nose on this full-bodied, tannic, heavily oaked Tinto Fino. Flavors of clove and woodspice sit on top of blackberry fruit, then hickory and other forceful oak-based aromas rise up and dominate the finish. | Tempranillo Blend |

In [16]:
# Query
query = "List items described with a fruit aroma, format answer as a table in markdown with columns; country, title, description and variety."
response = qa_chain.run(query)
display(Markdown(response))



> Entering new RetrievalQA chain...

> Finished chain.


| Country | Title | Description | Variety |
|---------|-------|-------------|---------|
| France  | Jean-Baptiste Adam 2012 Les Natures Pinot Gris (Alsace) | This has great depth of flavor with its fresh apple and pear fruits and touch of spice. It's off dry while balanced with acidity and a crisp texture. Drink now. | Pinot Gris |
| US      | Quiévremont 2012 Meritage (Virginia) | Red fruit aromas pervade on the nose, with cigar box and menthol notes riding in the back. The palate is slightly restrained on entry, but opens up to riper notes of cherry and plum specked with crushed pepper. This blend of Merlot, Cabernet Sauvignon and Cabernet Franc is approachable now and ready to be enjoyed. | Meritage |
| Argentina | Felix Lavaque 2010 Felix Malbec (Cafayate) | Baked plum, molasses, balsamic vinegar and cheesy oak aromas feed into a palate that's braced by a bolt of acidity. A compact set of saucy red-berry and plum flavors features tobacco and peppery accents, while the finish is mildly green in flavor, with respectable weight and balance. | Malbec |
| Italy   | Stemmari 2013 Dalila White (Terre Siciliane) | Pretty aromas of yellow flower and stone fruit lead the nose. The bright palate offers yellow apple, apricot, vanilla and delicate notes of lightly toasted oak alongside crisp acidity. | White Blend |

In [18]:
# Query
query = "Suggest me a spicy wine."
response = qa_chain.run(query)
print(response)



> Entering new RetrievalQA chain...

> Finished chain.
I would suggest the Leon Beyer 2012 Gewurztraminer (Alsace) from France. It is described as a dry wine with a spicy flavor and a tight, taut texture. It also has a strongly mineral character layered with citrus and pepper.
